# UDACITY Demo - Step Functions

In [5]:
import boto3
from sagemaker import get_execution_role

client = boto3.client('stepfunctions')

# Definitions are unweildly and also not valid python objects. Consider also using the Step interface
# for the services you wish to invoke, or just constructing a Step Function through the UI 
# and pasting the definition that is automatically generated. 

definition = """{
  "Comment": "A Hello World example of the Amazon States Language.",
  "StartAt": "Lambda Invoke (First)", 
  "States": {
    "Lambda Invoke (First)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-east-1:167257913878:function:lesson3-demo:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "Next": "Lambda Invoke (Second)"
    },
    "Lambda Invoke (Second)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-east-1:167257913878:function:lesson3-demo:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "End": true
    }
  }
}
"""

In [6]:
# Your SageMaker execution role needs to have both StepFunctions permission and IAM permission. This
# is seperate from the role passed in through 'roleArn' (You need IAM permissions specifically because
# you are handling other roles.) The roleArn that's passed into the state machine needs to have permissions
# to the services it's invoking. In this case, these are Lambda permissions. 

response = client.create_state_machine(
    name='boto3StateMachine3-demo2', # Names need to be unique. 
    definition=definition,
    roleArn='arn:aws:iam::167257913878:role/service-role/StepFunctions-MyStateMachine-Demo3-role-6971a4bb',
    type='STANDARD',
    loggingConfiguration={
        'level': 'OFF'
    }
)
print(response)

{'stateMachineArn': 'arn:aws:states:us-east-1:167257913878:stateMachine:boto3StateMachine3-demo2', 'creationDate': datetime.datetime(2023, 1, 14, 16, 19, 30, 251000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '5a3c03f6-c47b-4b54-9a7a-0a59b924a964', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5a3c03f6-c47b-4b54-9a7a-0a59b924a964', 'date': 'Sat, 14 Jan 2023 16:19:30 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '129'}, 'RetryAttempts': 0}}


In [7]:
response = client.start_execution(
    stateMachineArn='arn:aws:states:us-east-1:167257913878:stateMachine:boto3StateMachine3-demo2', # You can find this through the Console or through the 'response' object. 
    name='example1', # Execution names need to be unique within state machines. 
    input='{}' # Input needs to be at least empty brackets. 
)

In [8]:
print(response)

{'executionArn': 'arn:aws:states:us-east-1:167257913878:execution:boto3StateMachine3-demo2:example1', 'startDate': datetime.datetime(2023, 1, 14, 16, 20, 10, 690000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '9ebd097f-c7c6-4bac-afb9-26d8eb7ac677', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9ebd097f-c7c6-4bac-afb9-26d8eb7ac677', 'date': 'Sat, 14 Jan 2023 16:20:10 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '128'}, 'RetryAttempts': 0}}
